In [1]:
import os
os.chdir("../")

In [3]:
%pwd

'/Users/sidharthgoel/Desktop/deep_learning_disease_detection/Chickdis'

In [5]:
from Chickdis.utils.common import logger
from Chickdis.utils.common import create_directories,read_yaml
from Chickdis.constants import *

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [10]:
@dataclass 
class Callbacks_config:
  root_dir: Path
  tensorboard_root_log_dir: Path
  checkpoint_model_filepath: Path

In [33]:
class ConfigurationManager:
    def __init__(self,config_path=CONFIG_FILE_PATH,params_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_callback_config(self):
        config=self.config.prepare_callbacks
        create_directories([Path(config.checkpoint_model_filepath),Path(config.tensorboard_root_log_dir)])
        prepare_callback_config=Callbacks_config(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config

In [13]:
import urllib.request as request
import tensorflow as tf
import time

In [34]:
class PrepareCallbacks:
    def __init__(self,config:Callbacks_config):
        self.config=config
    
    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
            self.config.tensorboard_root_log_dir,f"tb-logs-at-{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    @property
    def _create_ckpt_callbacks(self):

        return tf.keras.callbacks.ModelCheckpoint(filepath=str(self.config.checkpoint_model_filepath),save_best_only=  True)
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,self._create_ckpt_callbacks
        ]

In [35]:
try:
    configuration_manager=ConfigurationManager()
    PrepareCallbacks_config=configuration_manager.get_callback_config()
    PrepareCallbacksobj=PrepareCallbacks(PrepareCallbacks_config)
    callback_list=PrepareCallbacksobj.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2025-09-08 15:14:20,691: INFO: common]: Directory created at: artifacts]
[2025-09-08 15:14:20,693: INFO: common]: Directory created at: artifacts/prepare_callbacks/checkpoint_model_filepath]


[2025-09-08 15:14:20,695: INFO: common]: Directory created at: artifacts/prepare_callbacks/tensorboard_root_log_dir]
